In [3]:
from lora.core import LORA, FUSE
from load_data import LoadData
from mlx_lm import generate, utils

In [4]:
# Configuration

root_folder = "model_a_1"

data_folder = f"./{root_folder}/data"
dataset_name = "n4jiDX/Math-Problems"
n = 1000
test_split_ratio = 0.2
valid_split_ratio = 0.2

model_path = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_file = f"./{root_folder}/adapters.npz"
save_model_path = f"./{root_folder}/model"

In [5]:
# Create Train, Test and Validation Data from the Dataset

system_message = """
You are a math problem solver. Given a math problem, you will provide a step-by-step solution.
Use the following format:
Problem: <The math problem>
Solution: <Step-by-step solution>
"""

def create_conversation(input: dict) -> dict:
    return {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": input["Problem"]},
            {"role": "assistant", "content": input["Solution"]}
        ]
    }

data_loader = LoadData(folder=data_folder, dataset_name=dataset_name)
data_loader.save(function=create_conversation, n=n, test_split_ratio=test_split_ratio, valid_split_ratio=valid_split_ratio, write_files=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'train': './model_a_1/data/train.jsonl',
 'test': './model_a_1/data/test.jsonl',
 'valid': './model_a_1/data/valid.jsonl'}

In [ ]:
# Fine-Tuning with LoRA

lora = LORA(config={"train": True, "batch_size": 1, "lora_layers": 4, "adapter_file": adapter_file})
lora.invoke(model_path=model_path, data=data_folder)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Total parameters 7242.158M
Trainable parameters 0.426M
Loading datasets
Training


In [ ]:
# Fuse the LoRA adapters with the base model and save the fine-tuned model

fuse = FUSE(config={"adapter_file": adapter_file})
fuse.invoke(model_path=model_path, save_path=save_model_path)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
# Load the fine-tuned model and generate a response

model, tokenizer = utils.load(save_model_path)
generate(model=model, tokenizer=tokenizer, prompt="You are a math problem solver. Given a math problem, you will provide a step-by-step solution.\nUse the following format:\nProblem: <The math problem>\nSolution: <Step-by-step solution>\nUser:Find the product of the solutions of the equation: $|y| = 3(|y| - 2)$.\nAssistant:")

'To solve this problem, we first need to find the solutions of the equation $|y| = 3(|y| - 2)$.\n\nSolving the equation $|y| = 3(|y| - 2)$ means finding the values of $y$ that satisfy the equation. Since $|y|$ is the absolute value of $y$, the equation is satisfied when $y$ is either 3 or $-3$ (the positive and negative solutions).\n\nNow, we need to find the product of these solutions:\n\n$$\n\\text{Product of solutions} = y_1 \\times y_2 \\\\\n= 3 \\times (-3) \\\\\n= -3 \\times 3 \\\\\n= -9\n$$\n\nSo, the product of the solutions of the equation $|y| = 3(|y| - 2)$ is $\\boxed{-9}$.'